# Random Numbers in SAS and Oracle 

We occasionally need to generate random numbers.  One common usage is to create a random sample of records in a table.  The examples below show ways to do that in SAS and Oracle.  There are, of course, many other ways to generate random numbers in SAS.

### The RAND and STREAMINIT functions in SAS

There have always been random number generators in SAS, but a new one was recently introduced to remedy some deficiencies in the traditional methods.

One old standard traditional method is the RANUNI function.  Here's what the current SAS documentation says about it:

> ___This function is deprecated.___ The function is suitable for small samples and for applications that do not require a sophisticated random-number generator. It is not suitable for parallel  and distributed processing. For more demanding applications, use the STREAMINIT subroutine and the RAND('Uniform') function. 

One suggested replacement with more statistical rigor is the RAND function.  Like the RANUNI function, it can return a stream of random numbers between 0 and 1, but it can also return integers within a range, normal distributions, binomial distributions, and more than a dozen other types of random number streams.

If you want a reproducible stream of numbers, you can use the CALL STREAMINIT function to specify a seed.

The example below also uses the STREAMREWIND function, which allows you to restart the random number stream and generate the same set of numbers two (or more) places in a data step.  You would not ordinarily use this function, but it makes the example easier to write.


In [1]:
data random_numbers_sas;

    call streaminit(94612);
    
    What = 'First Integer Stream    ';
    do i = 1 to 10;
        rand_num = rand('integer', 1, 100);
        output;
    end;
    
    What = 'Second Integer Stream';
    do i = 1 to 10;
        rand_num = rand('integer', 1, 100);
        output;
    end;
    
    What = 'Repeat Integer Stream';
    call streamrewind();
    do i = 1 to 10;
        rand_num = rand('integer', 1, 100);
        output;
    end;

    What = 'Uniform Stream';
    do i = 1 to 10;
        rand_num = rand('uniform');
        output;
    end;
    
    What = 'Normal Stream';
    do i = 1 to 10;
        rand_num = rand('normal');
        output;
    end;
    
run;
       
title 'Random Numbers from SAS';

proc report data=random_numbers_sas;
    columns ('Sequence' i) what , rand_num;
    define i        / group ' ';
    define what     / across order=data ' ';
    define rand_num / mean ' ';
run;    
    

SAS Connection established. Subprocess id is 19092



Sequence,First Integer Stream,Second Integer Stream,Repeat Integer Stream,Uniform Stream,Normal Stream
1,64,68,64,0.6702025,0.0459782
2,98,69,98,0.687109,-1.066737
3,52,98,52,0.9754485,-0.048917
4,83,48,83,0.4703581,-1.205843
5,65,55,65,0.5401,-1.129668
6,85,67,85,0.6676013,-0.137403
7,84,65,84,0.6440572,-0.158677
8,97,71,97,0.7098956,-0.124353
9,5,81,5,0.8059033,0.2708108
10,31,96,31,0.956014,1.1255771


### Use the DBMS_RANDOM package to create random numbers in Oracle

Use ``dbms_random.seed`` to set or reset the seed, and ``dbms_random.value`` to get a random value between 0 and 1. 

In [2]:
libname myora oracle path=dorrdb authdomain='DOR_DORRDB_AUTH';

%util_drop_tables(myora.random_numbers_oracle);

data myora.random_numbers_oracle;
    do i = 1 to 10;
        rand_num = . ;  /* We'll set the value in Oracle. */
        Pass = '1st Pass';
        output;
        Pass = '2nd Pass';
        output;
        Pass = '3rd Pass';
        output;
    end;
run;

proc sql stimer;

    connect to oracle as dorrdb (path=dorrdb authdomain='DOR_DORRDB_AUTH');
    
    execute( 
        exec dbms_random.seed(12345) 
           ) by dorrdb;    
    
    execute( 
        update random_numbers_oracle
            set rand_num = dbms_random.value() 
        where Pass = '1st Pass'
           ) by dorrdb;

    execute( 
        update random_numbers_oracle
            set rand_num = dbms_random.value() 
        where Pass = '2nd Pass'
           ) by dorrdb;
    
    execute( 
        exec dbms_random.seed(12345) 
           ) by dorrdb;    
    
    execute( 
        update random_numbers_oracle
            set rand_num = dbms_random.value() 
        where Pass = '3rd Pass'
           ) by dorrdb;
   
   disconnect from dorrdb;
               
quit;    

      
title 'Random Numbers from Oracle';

proc report data=myora.random_numbers_oracle;
    columns ('Sequence' i) pass , rand_num;
    define i        / group ' ';
    define pass     / across ' ';
    define rand_num / mean ' ' format=10.8;
run;   

Sequence,1st Pass,2nd Pass,3rd Pass
1,0.07686996,0.58820200,0.07686996
2,0.35785947,0.54262067,0.35785947
3,0.91345492,0.94949181,0.91345492
4,0.04033835,0.60687092,0.04033835
5,0.29942034,0.61047907,0.29942034
6,0.74329238,0.15005571,0.74329238
7,0.94622290,0.41306811,0.94622290
8,0.68817148,0.06352448,0.68817148
9,0.06209068,0.66017738,0.06209068
10,0.17257625,0.70760813,0.17257625
